In [1]:
import requests
from bs4 import BeautifulSoup as b4
import pandas as pd
import time

mobile_names = []
ratings = []
no_reviews = []
bought_last_month = []
mobile_price = []

for j in range(1, 51):
    time.sleep(2)  # Pause to reduce chances of getting blocked

    url = f"https://www.amazon.com/s?k=phones&page={j}&xpid=JlnAWWhUg8SUy&crid=34IIB9Q6BYDU8&qid=1751122090&sprefix=%2Caps%2C359&ref=sr_pg_2"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/125.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    r = requests.get(url, headers=headers)
    
    # Check for CAPTCHA or block
    if "captcha" in r.text.lower() or "Enter the characters you see below" in r.text:
        print(f"[Blocked] CAPTCHA triggered on page {j}")
        continue

    soup = b4(r.text, "lxml")
    containers = soup.find_all("div", {"data-component-type": "s-search-result"})
    print(f"[Page {j}] Found {len(containers)} products")

    for c in containers:
        title = c.find("h2")
        mobile_names.append(title.text.strip() if title else "none")

        rating = c.find("a", class_="a-popover-trigger a-declarative")
        if rating and rating.get("aria-label"):
            r_rating = rating.get("aria-label")
            ratings.append(r_rating.strip()[0:18])
        else:
            ratings.append("none")

        number_of_reviews = c.find("span", class_="a-size-base s-underline-text")
        no_reviews.append(number_of_reviews.text.strip() if number_of_reviews else "none")

        last = c.find("span", class_="a-size-base a-color-secondary")
        bought_last_month.append(last.text.strip() if last else "none")

        price = c.find("span", class_="a-price")
        mobile_price.append(price.text.strip() if price else "none")

# Create DataFrame
df = pd.DataFrame({
    "Mobile Name": mobile_names,
    "Rating": ratings,
    "Number of Reviews": no_reviews,
    "Bought Last Month": bought_last_month,
    "Price": mobile_price
})

print(df)
df.to_csv("amazon_mobile_phones1.csv")

[Page 1] Found 16 products
[Page 2] Found 22 products
[Page 3] Found 22 products
[Page 4] Found 22 products
[Page 5] Found 22 products
[Page 6] Found 16 products
[Page 7] Found 16 products
[Page 8] Found 16 products
[Page 9] Found 16 products
[Page 10] Found 16 products
[Page 11] Found 16 products
[Page 12] Found 22 products
[Page 13] Found 16 products
[Page 14] Found 16 products
[Page 15] Found 22 products
[Page 16] Found 16 products
[Page 17] Found 16 products
[Page 18] Found 22 products
[Page 19] Found 22 products
[Page 20] Found 2 products
[Page 21] Found 6 products
[Page 22] Found 6 products
[Page 23] Found 6 products
[Page 24] Found 6 products
[Page 25] Found 0 products
[Page 26] Found 0 products
[Page 27] Found 6 products
[Page 28] Found 6 products
[Page 29] Found 6 products
[Page 30] Found 0 products
[Page 31] Found 6 products
[Page 32] Found 0 products
[Page 33] Found 0 products
[Page 34] Found 0 products
[Page 35] Found 6 products
[Page 36] Found 0 products
[Page 37] Found 0 

In [2]:
#for some reason the price elements are repeated twice so this script solves it

import pandas as pd

# Load the CSV file
file_path = "amazon_mobile_phones1.csv"  
df = pd.read_csv(file_path)

def clean_price(price):
    if isinstance(price, str) and price.count("$") > 1:
        return price[:price.find("$", price.find("$") + 1)]
    return price

# Apply the function to the 'Price' column
df['Price'] = df['Price'].apply(clean_price)

# Optionally, save the cleaned data to a new CSV file
df.to_csv("cleaned_amazon_mobile_phones.csv", index=False)

# Display the first few cleaned rows
print(df[['Mobile Name', 'Price']])


                                           Mobile Name    Price
0    Samsung Galaxy A16 5G A Series, Unlocked Andro...  $174.99
1    Tracfone | Motorola Moto g Play 2024 | Locked ...   $39.88
2    Motorola Moto G - 2025 | Unlocked | Made for U...  $199.99
3    Samsung Galaxy S24 FE AI Phone, 256GB Unlocked...  $689.69
4    Samsung Galaxy S25+ Cell Phone, 256GB Smartpho...     none
..                                                 ...      ...
427  SIELO 2 Pack Matte Screen Protector, 9H Hardne...     none
428  Big Button Phone for Elderly, JeKaVis J-P02 Am...     none
429  MMY I25 Ultra 5G Unlocked Smartphone,8GB+256GB...     none
430  2025 New 45W Super Fast Charger USB C Android ...     none
431  2 Pack Matte Privacy Screen Protector for iPho...     none

[432 rows x 2 columns]
